# Purpose of this notebook

The purpose of this notebook is to help developers setup, configure and test the Retrieve and Rank service for the enhanced dar demonstration. 

# Create a cluster

The R&R service works by retrieving and ranking documents using Apache Solr and Machine Learning algorithms. In order to use the service, a cluster needs to be created which will store the documents that will be retrieved and ranked.

    NOTE: In this step you may want to customize the values for the `CLUSTER_NAME` and `CLUSTER_SIZE` constants. The sample code contains suggested defaults.
    
To create a cluster, run the following code:

In [1]:
import subprocess
import json
import shlex
import os
from watson_developer_cloud import RetrieveAndRankV1


#getting current directory
curdir = os.getcwd()

#loading credentials
credFilePath = curdir+'/../config/car_enhanced_credentials.json'
credentials = ''
with open(credFilePath) as credFile:
    credentials = json.load(credFile)

BASEURL=credentials['url']
SOLRURL= BASEURL+"solr_clusters/"
USERNAME=credentials['username']
PASSWORD=credentials['password']

#please provide the name and size of the cluster below
CLUSTER_NAME="rr_skbootcamp_car_cluster"
CLUSTER_SIZE=1  #Size of the cluster to create. Ranges from 1 to 7. 
                #Send an empty value to create a small free cluster for testing. 
                #You can create only one free cluster.

#Checking if cluster with same name already exists
retrieve_and_rank = RetrieveAndRankV1(
    username=USERNAME,
    password=PASSWORD)
found = False
cluster_id = ''
output = retrieve_and_rank.list_solr_clusters()
try:
    parsed_json = output
    clusters = parsed_json['clusters']
    for i in range(len(clusters)):
        cluster_json = clusters[i]
        if cluster_json['cluster_name'] == CLUSTER_NAME:
            found = True
            cluster_id = cluster_json['solr_cluster_id']
except:
    print ('Command:')
    print ('retrieve_and_rank.list_solr_clusters()')
    print ('Response:')
    print (output) 

if found:
    print "Cluster "+CLUSTER_NAME+" already exists with ID "+cluster_id+"."
    print json.dumps(parsed_json, sort_keys=True, indent=4)
else:
    #Running command that creates a cluster
    output = retrieve_and_rank.create_solr_cluster(cluster_name=CLUSTER_NAME,cluster_size=CLUSTER_SIZE)

    try:
        parsed_json = output
        print json.dumps(parsed_json, sort_keys=True, indent=4)
        credentials['cluster_id'] = parsed_json['solr_cluster_id']
        with open(credFilePath, 'w') as credFileUpdated:
            json.dump(credentials, credFileUpdated)
    except:
        print ('Command:')
        print ('retrieve_and_rank.create_solr_cluster')
        print ('Response:')
        print (output)  

{
    "cluster_name": "rr_skbootcamp_car_cluster", 
    "cluster_size": "1", 
    "solr_cluster_id": "scbf1ac122_26d0_4d8c_b747_127a7f04db3f", 
    "solr_cluster_status": "NOT_AVAILABLE"
}


If you see an error message about invalid credentials, make sure that you modified the file `config/car_enhanced_credentials.json`, as specified in the `README.md` file for this repository. If the cluster is created successfully, the json response will show the status of the cluster as `NOT_AVAILABLE`. It takes a few minutes for the cluster to show the status `READY`. To check on the status of the cluster, run the code below.

# Check the Status of a Cluster

### Note : 
You have to wait for the cluster status to become "Ready" before proceeding the next section for creating a solr configuration.

In [7]:
import subprocess
import json
import shlex
import os
from watson_developer_cloud import RetrieveAndRankV1

#getting current directory
curdir = os.getcwd()

credFilePath = curdir+'/../config/car_enhanced_credentials.json'
credentials = ''
with open(credFilePath) as credFile:
    credentials = json.load(credFile)
USERNAME=credentials['username']
PASSWORD=credentials['password']
#get cluster id from the loaded credentials in Step 1
CLUSTER_ID=credentials['cluster_id']

retrieve_and_rank = RetrieveAndRankV1(
    username=USERNAME,
    password=PASSWORD)

#Running command that checks the status of a cluster
output = retrieve_and_rank.get_solr_cluster_status(solr_cluster_id=CLUSTER_ID)

try:
    parsed_json = json.loads(output)
    print json.dumps(parsed_json, sort_keys=True, indent=4)
except:
    print ('Command:')
    print ('retrieve_and_rank.get_solr_cluster_status')
    print ('Response:')
    print (output)
    

Command:
retrieve_and_rank.get_solr_cluster_status
Response:
{u'cluster_name': u'rr_skbootcamp_car_cluster', u'solr_cluster_status': u'READY', u'cluster_size': u'1', u'solr_cluster_id': u'scbf1ac122_26d0_4d8c_b747_127a7f04db3f'}


# Content

The next few sections demonstrate the steps necessary to create a Solr collection, which is where indexed data is stored.

After you have created your Solr documents file, add your file to the data/content directory and name it car_enhanced_solrDocuments.json so they can be used to populate the cluster in the next steps of this notebook.

## Upload a Solr Configuration

Since each corpus (collection of documents) might contain documents with different fields, it is needed to configure Solr so that it knows which fields should be stored and indexed for retrieval. You can also augment a set of synonyms related to your domain to improve the performance of the retrieval phase and add that to the configuration, for example.

The config file (`schema.xml`) that is appropriate for the data being used in this tutorial can be found in the config folder downloaded from the repository. You should modify this file to meet the criteria of your data.

After you have modified and compacted the configuration files, set the variable `CONFIG_FILE_NAME` in the next section of code to the name of your configuration file. Running the code below uploads the configuration to Solr.

- NOTE 1: Ensure that you assign type="watson_text_en" for any new fields you add to schema.xml else they won't be recognized as Watson features and used by the ranker.

- NOTE 2: Don't zip the config folder itself but instead select the contents of the config folder and zip just them (including subfolders).  Failing to do this will prevent your config.zip from being properly recognized.

- NOTE 3: In this step you will need to provide the values for the `CONFIG_NAME` and `CONFIG_FILE_NAME` constants.


### Copy the provided car_enhanced_solrconfig.zip to the config directory


In [8]:
import subprocess
import json
import shlex
import os
from watson_developer_cloud import RetrieveAndRankV1

#getting current directory
curdir = os.getcwd()

#load the credentials
credFilePath = curdir+'/../config/car_enhanced_credentials.json'
credentials = ''
with open(credFilePath) as credFile:
    credentials = json.load(credFile)
USERNAME=credentials['username']
PASSWORD=credentials['password']

#getting relative path to config file
CONFIG_PATH=curdir+"/../config/"

#get cluster id from the loaded credentials in Step 1
SOLR_CLUSTER_ID=credentials['cluster_id']
found = False

#please provide the configuration name and and the path to the zip file
CONFIG_NAME="rr_skbootcamp_car_config"   #Do not add spaces or special characters
CONFIG_FILE_NAME="car_enhanced_solrconfig.zip" #Change this if you renamed the file

retrieve_and_rank = RetrieveAndRankV1(
    username=USERNAME,
    password=PASSWORD)

# Check if config exists
output = retrieve_and_rank.list_configs(solr_cluster_id=SOLR_CLUSTER_ID)
try:
    configs = output['solr_configs']
    for i in range(len(configs)):
        config_name = configs[i]
        if config_name == CONFIG_NAME:
            found = True
except:
    print ('Command:')
    print ('retrieve_and_rank.list_solr_clusters()')
    print ('Response:')
    print (output) 

if found:
    print "Configuration "+CONFIG_NAME+" already exists."
    print json.dumps(output, sort_keys=True, indent=4)
else:
    #Running command that uploads a configuration to solr
    with open(CONFIG_PATH + CONFIG_FILE_NAME, 'rb') as config:
        output = retrieve_and_rank.create_config(SOLR_CLUSTER_ID, CONFIG_NAME, config)

    try:
        print json.dumps(output, sort_keys=True, indent=4)
        credentials['config_name'] = CONFIG_NAME
        with open(credFilePath, 'w') as credFileUpdated:
            json.dump(credentials, credFileUpdated)
    except:
        print ('Command:')
        print ('retrieve_and_rank.create_config()')
        print ('Response:')
        print (output)  


{
    "message": "WRRCSR026: Successfully uploaded named config [rr_skbootcamp_car_config] for Solr cluster [scbf1ac122_26d0_4d8c_b747_127a7f04db3f].", 
    "statusCode": 200
}


## Create a Collection

After the configuration has been uploaded to Solr, the cluster is ready to receive collections. Create a new collection which will store the documents by running the code below. You need to choose a name for the collection.

    NOTE: In this step you will need to provide the value for the `COLLECTION_NAME` constant.

In [9]:
import subprocess
import json
import shlex
import os
from watson_developer_cloud import RetrieveAndRankV1

#getting current directory
curdir = os.getcwd()

#load the credentials
credFilePath = curdir+'/../config/car_enhanced_credentials.json'
credentials = ''
with open(credFilePath) as credFile:
    credentials = json.load(credFile)
USERNAME=credentials['username']
PASSWORD=credentials['password']

#get cluster id and config name from the loaded credentials in Step 1
SOLR_CLUSTER_ID=credentials['cluster_id']
CONFIG_NAME=credentials['config_name']
found = False

#please provide the collection name
COLLECTION_NAME="rr_skbootcamp_car_collection"   #Do not add spaces or special characters

retrieve_and_rank = RetrieveAndRankV1(
    username=USERNAME,
    password=PASSWORD)

#Check to see if collection exists
output = retrieve_and_rank.list_collections(solr_cluster_id=SOLR_CLUSTER_ID)

try:
    collections = output['collections']
    for i in range(len(collections)):
        collection_name = collections[i]
        if collection_name == COLLECTION_NAME:
            found = True
except:
    print ('Command:')
    print ('retrieve_and_rank.list_collections')
    print ('Response:')
    print (output) 
    
if found:
    print "Collection "+COLLECTION_NAME+" already exists."
    print json.dumps(output, sort_keys=True, indent=4)
else:
    #Running command to create a new collection
    output = retrieve_and_rank.create_collection(SOLR_CLUSTER_ID, COLLECTION_NAME, CONFIG_NAME)
    try:
        print json.dumps(output, sort_keys=True, indent=4)
        credentials['collection_name'] = COLLECTION_NAME
        with open(credFilePath, 'w') as credFileUpdated:
            json.dump(credentials, credFileUpdated)
    except:
        print ('Command:')
        print ('retrieve_and_rank.create_collection()')
        print ('Response:')
        print (output)

{
    "responseHeader": {
        "QTime": 10920, 
        "status": 0
    }, 
    "success": {
        "10.176.143.179:5238_solr": {
            "core": "rr_skbootcamp_car_collection_shard1_replica1", 
            "responseHeader": {
                "QTime": 2635, 
                "status": 0
            }
        }, 
        "10.176.42.206:6215_solr": {
            "core": "rr_skbootcamp_car_collection_shard1_replica2", 
            "responseHeader": {
                "QTime": 2294, 
                "status": 0
            }
        }
    }
}


## Populate the Collection

After the collection has been created, it is ready to store and index documents. The command below uploads the documents in the file mentioned on step #3.

### Copy the provided car_enhanced_solrDocuments.json file to the content directory


In [10]:
import subprocess
import json
import shlex
import os
import pysolr
from watson_developer_cloud import RetrieveAndRankV1

#getting current directory
curdir = os.getcwd()

#get cluster id and collection name from the loaded credentials in Step 1
BASEURL=credentials['url']
SOLRURL= BASEURL+"solr_clusters/"
USERNAME=credentials['username']
PASSWORD=credentials['password']
SOLR_CLUSTER_ID=credentials['cluster_id']
COLLECTION_NAME=credentials['collection_name']

retrieve_and_rank = RetrieveAndRankV1(
    username=USERNAME,
    password=PASSWORD)

#please make sure your documents are in data/content/solrDocuments.json or modify the path below
SOLR_DOCUMENTS_PATH=curdir+"/../data/content/car_enhanced_solrDocuments.json"

#Get Solr Client Handle
pysolr_client = retrieve_and_rank.get_pysolr_client(SOLR_CLUSTER_ID, COLLECTION_NAME)

#Add documents
with open(SOLR_DOCUMENTS_PATH) as data_file:    
    data = json.load(data_file)
output = pysolr_client.add(data)

#Running command that index documents
try:
    print (output)
except:
    print ('Command:')
    print ('pysolr_client.add()')
    print ('Response:')
    print (output)

<?xml version="1.0" encoding="UTF-8"?>
<response>
<lst name="responseHeader"><int name="status">0</int><int name="QTime">982</int></lst>
</response>



## Test Solr Retrieval

At this point the Solr cluster is configured and ready to retrieve documents based on a query. To test it, submit a query to it by running the command below. A list of 10 documents should return from the query.

    NOTE: In this step you should provide the values for the QUESTION and MAX_DOCUMENTS constants.

In [11]:
import subprocess
import json
import shlex
import os
import pysolr
from watson_developer_cloud import RetrieveAndRankV1

#getting current directory
curdir = os.getcwd()

#loading credentials
credFilePath = curdir+'/../config/car_enhanced_credentials.json'
with open(credFilePath) as credFile:
    credentials = json.load(credFile)

BASEURL=credentials['url']
SOLRURL= BASEURL+"solr_clusters/"
USERNAME=credentials['username']
PASSWORD=credentials['password']
SOLR_CLUSTER_ID=credentials['cluster_id']
COLLECTION_NAME=credentials['collection_name']

retrieve_and_rank = RetrieveAndRankV1(
    username=USERNAME,
    password=PASSWORD)

#please provide the query to test
QUESTION="How to check tire pressure"

#please provide the number of documents the query should return
MAX_DOCUMENTS=10

#Running command that queries Solr
QUESTION = QUESTION.replace(" ","%20")

#Invoke Solr Search using python Solr client
pysolr_client = retrieve_and_rank.get_pysolr_client(SOLR_CLUSTER_ID, COLLECTION_NAME)
output = pysolr_client.search('*',fq=QUESTION,rows=10)

try:
    print (output.docs)
except:
    print ('Command:')
    print ('pysolr_client.search()')
    print ('Response:')
    print (output)

[{u'body': u"Use a child safety seat (sometimes called an infant carrier, convertible seat, or toddler seat) for infants, toddlers, or children weighing 40 pounds (18 kilograms) or less (generally age four or younger).<br><br>Perform the following steps when installing the child seat with combination lap and shoulder belts:<br><br>We recommend checking with a NHTSA Certified Child Passenger Safety Technician to make certain the child restraint is properly installed. In Canada, check with Transport Canada for referral to a Child Car Seat Clinic.<br><br>We recommend checking with a NHTSA Certified Child Passenger Safety Technician to make certain the child restraint is properly installed. In Canada, check with Transport Canada for referral to a Child Car Seat Clinic.<br><br>The LATCH system is composed of three vehicle anchor points: two lower anchors located where seat back and seat cushion meet (called the seat bight) and one top tether anchor located behind that seating position.<br><

If the query returns documents successfully, the next step is to create a ground truth file to train a ranker of the results. This will improve the results returned for a given query.

## Train the Ranker

Run the command below to train a ranker. This command first generates a training data file containing feature vectors for each one of the questions. When the file is generated, it is then sent to the ranker for training. You should see a (long!) feedback printed below the code cell once the ranker starts being trained. You can then check the status of the ranker by running step 11.

    NOTE: In this step you will need to provide the value for the RANKER_NAME constant.
    
### Copy the pre-created training data file - car_enhanced_trainingdata.csv


In [12]:
import subprocess
import json
import shlex
import os
from watson_developer_cloud import RetrieveAndRankV1


#getting current directory
curdir = os.getcwd()

#loading credentials
credFilePath = curdir+'/../config/car_enhanced_credentials.json'
with open(credFilePath) as credFile:
    credentials = json.load(credFile)
USERNAME=credentials['username']
PASSWORD=credentials['password']
TRAINING_DATA=curdir+'/../data/groundtruth/car_enhanced_trainingdata.csv'

#please provide the ranker name
RANKER_NAME="rr_skbootcamp_car_ranker"

retrieve_and_rank = RetrieveAndRankV1(
    username=USERNAME,
    password=PASSWORD)

#Checking if ranker with same name already exists
found = False
ranker_id = ''
output = retrieve_and_rank.list_rankers()
try:
    rankers =  output['rankers']
    for i in range(len(rankers)):
        ranker_json = rankers[i]
        if ranker_json['name'] == RANKER_NAME:
            found = True
            ranker_id = ranker_json['ranker_id']
except:
    print ('Command:')
    print ('list_rankers()')
    print ('Response:')
    print (output) 

if found:
    print "Ranker "+RANKER_NAME+" already exists with ID "+ranker_id+"."
    print json.dumps(output, sort_keys=True, indent=4)
else:
    #Running command that trains a ranker
    with open(TRAINING_DATA, 'rb') as training_data:
        output = retrieve_and_rank.create_ranker(training_data=training_data, name=RANKER_NAME)
    try:
        print json.dumps(output, sort_keys=True, indent=4)
        credentials['ranker_id'] = output['ranker_id']
        with open(credFilePath, 'w') as credFileUpdated:
            json.dump(credentials, credFileUpdated)
    except:
        print ('Command:')
        print (cmd)
        print ('Response:')
        print (output)

{
    "created": "2016-11-16T21:20:57.811Z", 
    "name": "rr_skbootcamp_car_ranker", 
    "ranker_id": "c852c8x19-rank-3622", 
    "status": "Training", 
    "status_description": "The ranker instance is in its training phase, not yet ready to accept rank requests", 
    "url": "https://gateway.watsonplatform.net/retrieve-and-rank/api/v1/rankers/c852c8x19-rank-3622"
}


## Check Status of Ranker

In [24]:
import subprocess
import json
import shlex
import os
from watson_developer_cloud import RetrieveAndRankV1

#getting current directory
curdir = os.getcwd()

#loading credentials
credFilePath = curdir+'/../config/car_enhanced_credentials.json'
with open(credFilePath) as credFile:
    credentials = json.load(credFile)
    
RANKER_ID=credentials['ranker_id']
USERNAME=credentials['username']
PASSWORD=credentials['password']

retrieve_and_rank = RetrieveAndRankV1(
    username=USERNAME,
    password=PASSWORD)

#Running command that checks the status of a ranker
output = retrieve_and_rank.get_ranker_status(RANKER_ID)
try:
    #parsed_json = json.loads(output)
    print json.dumps(output, sort_keys=True, indent=4)
except:
    print ('Command:')
    print ('retrieve_and_rank.get_ranker_status()')
    print ('Response:')
    print (output)

{
    "created": "2016-11-16T21:20:57.811Z", 
    "name": "rr_skbootcamp_car_ranker", 
    "ranker_id": "c852c8x19-rank-3622", 
    "status": "Training", 
    "status_description": "The ranker instance is in its training phase, not yet ready to accept rank requests", 
    "url": "https://gateway.watsonplatform.net/retrieve-and-rank/api/v1/rankers/c852c8x19-rank-3622"
}


## Test the Ranker

To test the ranker, submit a query and observe the results returned by running the command below

    NOTE: In this step you should provide the value for the QUESTION constant.

In [ ]:
import subprocess
import json
import shlex
import os

#getting current directory
curdir = os.getcwd()

#loading credentials
credFilePath = curdir+'/../config/car_enhanced_credentials.json'
with open(credFilePath) as credFile:
    credentials = json.load(credFile)

BASEURL=credentials['url']
SOLRURL= BASEURL+"solr_clusters/"
RANKER_URL=BASEURL+"rankers"
USERNAME=credentials['username']
PASSWORD=credentials['password']
SOLR_CLUSTER_ID=credentials['cluster_id']
COLLECTION_NAME=credentials['collection_name']
RANKER_ID=credentials['ranker_id']

#please provide the query to test
QUESTION="How to check tire pressure"

#Running command that queries Solr
QUESTION = QUESTION.replace(" ","%20")
curl_cmd = 'curl -u "%s":"%s" "%s%s/solr/%s/fcselect?ranker_id=%s&q=%s&wt=json&fl=id,title,body,sourceUrl"' %\
(USERNAME, PASSWORD, SOLRURL, SOLR_CLUSTER_ID, COLLECTION_NAME, RANKER_ID, QUESTION)
process = subprocess.Popen(shlex.split(curl_cmd), stdout=subprocess.PIPE)
output = process.communicate()[0]
try:
    parsed_json = json.loads(output)
    print json.dumps(parsed_json, sort_keys=True, indent=4) 
except:
    print ('Command:')
    print (curl_cmd)
    print ('Response:')
    print (output)

You have reached the end of the Retrieve and Rank configuration. The next step is to run an experiment to compare your test set using Solr and the newly trained ranker.